In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
pd.set_option('display.max_columns',100)

In [2]:
# credentials
username = os.environ['DBA_USR']
password = os.environ['DBA_PW']
dsn = '10.220.50.121:1661/ANALYTIC'

In [3]:
# create connection
connection = oracledb.connect(user=username, password=password, dsn=dsn)

In [4]:
# create engine
engine = create_engine('oracle+oracledb://', creator=lambda: connection)

In [ ]:
# open and read the file as a single buffer
sqlQuery = f'''SELECT AGREE_ID, GCIF_NO, CUST_NAME, CIF_NO, NOTE_NO, PRD_TP, 
PRD_NM,PRD_SEGMENT, ORG_LMT_AMT, CURR_LMT_AMT, BAL, ORIG_INT_RT, CURR_INT_RT, OTH_CURR_INT_RT,
CLCT_RTNG_FCL, ALLOW_PCT, ALLOW_PCT_ADD, RESTRUCT_CD, PASTDUE_SINCE_DT, PASTDUE_DAYS, WRITE_OFF_AMT,
WRITE_OFF_DT, WRITE_OFF_YN  from PDA.MASTER_LOAN
WHERE BASE_DT = {date}
AND PRD_NM LIKE '%Credit Card%'
AND STATUS = '00001''''

In [ ]:
# run query and store data in df
ML = pd.read_sql(sqlQuery, engine)

In [ ]:
# change column names into UPPER CASE
ML.columns = [x.upper() for x in ML.columns]